# Reading and writing files, JSON and CSV (with Pandas)

## Contents:

* File Input/Output
* Reading and writing JSON
* Reading and writing CSV (with Pandas)

## File Input/Output

A huge portion of our input data will come from files that we have stored on our computer (on the file system). A lot of analysis of these files is done in memory in Python, when working with them. We have to save them back to the file system to store the results. So, mastering the art of reading and writing is crucial in programming.

Until now, we have run stuff (almost instantly) in our Jupyter Notebooks, but imagine that we write code that takes a couple of ours to run on a large collection of files. Then we want to save the result, either for further analysis, or to make these files available (i.e. sharing) in your research. 

The following code opens a file in our filesystem, prints the first 10 lines and closes the file. Please note that this file must exist on your computer. If you only have downloaded this notebook, go back to the repository, download the file, and place it in the appropriate path (or change the path below). This path corresponds to the folder structure on your file system. 

> **Please note:** The code below shows you how the `open()` function works. It's better to use a `width` block (see below), which does this opening and closing for you.

In [ ]:
infile = open('data/adams-hhgttg.txt', 'r', encoding='utf-8')

for i, line in enumerate(infile):
    if i == 10:
        break
    print(line)

infile.close()

The key passage here is the one in which the `open()` function opens a file and return a **file object** (hint: try printing the type of `infile`), and it is commonly used with the following three parameters: the **name of the file** that we want to open, the **mode** and the **encoding**. 

- **filename**: the name of the file to open, this corresponds to the full/relative path to the file from the notebook. 

- the **mode** in which we want to open a file: the most commonly used values are `r` for **reading** (default, which means that you don't have to put this in explicitly), `w` for **writing** (overwriting existing files), and `a` for **appending**. (Note that [the documentation](https://docs.python.org/3/library/functions.html#open) report mode values that may be necessary in some exceptional case)

- **encoding**: which mapping of string to code points (conversion to bytes) to use, more on this later. 

>**IMPORTANT**: every opened file should be **closed** by using the function `close()` before the end of the program, or the file could be unavailable to successive manipulations or for other programs.

There are other ways to read a text file, among which the use of the methods `read()` and `readlines()`, that would simplify the above function in:

```python
infile = open('data/adams-hhgttg.txt', 'r', encoding='utf-8')
text = infile.readlines()
print(text[:10])
infile.close()
```

However, these methods **read the whole file at once**, thus creating capacity/efficiency problems when working with big corpora.

In the solution we adopt here the input file is read line by line, so that at any given moment **only one line of text** is loaded into memory. 

You can see all file object methods, including examples, on this W3schools page: https://www.w3schools.com/python/python_ref_file.asp

### The with statement 

A `with` statement is used to wrap the execution of a block of code.

Using this construction to open files has three major advantages:

- there is no need to explicitly  close the file (the file is automatically closed as soon as the nested code exits)
- the file is closed automatically even when unhandled errors cause the program to crash
- the code is way clearer (it is trivial to identify where in the code a file is opened) 

Thus, you can  make it yourself a bit easier. Forget about the explicit `.close()` method. The code above can be rewritten as follows:

In [ ]:
with open('data/adams-hhgttg.txt', encoding='utf-8') as infile:  # The file is opened
    
    lines = infile.readlines()
    
# As soon as we exit the indented scope, the file is closed again 
# (and made available to other programs on your computer)
print(lines[:10])

The code in the indented with block is executed while the file is opened. It is automatically closed as the block is closed. 

### Quiz

Hint: you can call `.read()` on the file object.

* Write one function that takes a file path as argument and prints statistics about the file, giving:
    * The number of words (often called 'tokens')
    * The number of unique words (often called 'types')
    * The type:token ratio (i.e. unique words / words)
    * The 10 most frequent words, including their frequencies
* Write a normalization or cleaning function that takes a string as argument, that pre-processes this text and returns a normalized version, by removing/substituting:
    * Uppercase characters
    * Punctuation
* Call the normalization function inside the first function

Test the function on the filepath in `file_path` below. Compare the results from running the function with and without normalization.

In [ ]:
# Your code here

In [ ]:
file_path = 'data/adams-hhgttg.txt'

# your_function_name(file_path)

---

## Writing files

Writing an output file in Python has a structure that is close to that we're used in our reading examples above. The main difference are:

- the specification of the **mode** `w`
- the use of the function `write()` for each line of text

> **Warning!** Opening an _existing_ file in `w` mode will erase its contents!

In [ ]:
# The folder you with to write the file to ('stuff' below) has to exist on the file system

with open('stuff/output-test-1.txt', 'w', encoding='utf-8') as outfile:
    
    outfile.write("My name is:")
    outfile.write("John")

When writing line by line, it's up to you to take care of the **newlines** by appending `\n` to each line. Unlike the `print()` function, the `write()` function has no standard line-end character.

In [ ]:
with open('stuff/output-test-2.txt', 'w', encoding='utf-8') as outfile:
    
    outfile.write("My name is:\n")
    outfile.write("Alexander")


We can inspect the file we just created with the command line. The following is not Python, but a basic command line tool to print the contents of a file. At least on Mac and Linux, this works. Otherwise, just navigate to the file in your file explorer and open it.

> Prepending a `!` to a command executes a program on your computer. Use it with care and don't run such a cell in a notebook that you do not trust!

In [ ]:
!cat stuff/output-test-2.txt

### Quiz

Instead of printing the statistics in the previous quiz, write them to a file. For instance, use the file path in `file_path` to write the file to. Copy your function from above, rename it and add the required code to it.


In [ ]:
# Your code here

file_path = 'stuff/adams-hhgttg-statistics.txt'

# your_adapted_function_that_writes_statistics(file_path)


Let's quickly check its contents:

In [ ]:
!cat stuff/adams-hhgttg-statistics.txt

---

## Looping through folders and files

If you want to load in multiple files in a folder, without explicitly providing the file pointers/paths for each file, you can also point to a folder. We can use the built-in `os` module to loop through a folder and load multiple files in memory.

In [ ]:
import os  # You only have to do this once in your code. 
           # Always put this at the top of your file.

In [ ]:
gutenberg_books = dict()  # Create an empty dictionary to store our data in

for root, dirs, files in os.walk("data/gutenberg-extension"):
    for file in files:
        
        if not file.endswith('.txt'):  # Why this?
            continue
        
        # You have to specify the full (relative) path, not only the file name.
        file_path = os.path.join(root, file)  
        
        with open(file_path, encoding='utf-8') as infile:
            gutenberg_books[file] = infile.read()

In [ ]:
gutenberg_books.keys()

The `os.walk()` method is convenient if you are dealing with a combination of files and folders, no matter how deep the hierarchy goes (folders in folders etc.). A simpler function is `os.listdir()`.

In [ ]:
gutenberg_books = dict()  # Create an empty dictionary to store our data in

folder_path = "data/gutenberg-extension"

for file in os.listdir(folder_path):

    if not file.endswith('.txt'):  # Why this?
        continue
    
    file_path = os.path.join(folder_path, file)
    
    with open(file_path, encoding='utf-8') as infile:
        gutenberg_books[file] = infile.read()

In [ ]:
gutenberg_books.keys()

The dictionary object now contains a lot of information: all the contents of all files. There's a chance that your browser/notebook will crash when calling the dictionary here. Instead, let's call a part of one of the books, the first 300 characters:

In [ ]:
print(gutenberg_books['doyle-sherlock.txt'][:300])

---

# Reading and writing data in JSON and CSV

We now know how we can read and write textual content to files on our file system. Two more structed and common data formats to store data in are JSON and CSV. If you are not familiar with these, take a look at:

* JSON (https://www.w3schools.com/whatis/whatis_json.asp)
* CSV (https://www.howtogeek.com/348960/what-is-a-csv-file-and-how-do-i-open-it/)

## JSON

The syntax of JSON is very similar to the syntax of `int`, `str`, `list` and `dict` data types in Python. 

The following data (excerpt) is taken from the data that feeds the Instagram page of the UvA (https://www.instagram.com/uva_amsterdam/). The API/service of Instagram returns web data in JSON that is used by your browser to show you a page with content. You can also find this when inspecting the source of the page. 

A JSON file (named `example.json`) that looks like this:
```json
{
    "biography": "Welcome to the UvA \u274c\u274c\u274c \nFind out more about our:\n\ud83c\udfdb campuses \ud83c\udf93 education \ud83d\udd0e research\nShare your \ud83d\udcf8 using: #uva_amsterdam\nQuestions? Contact us:",
    "blocked_by_viewer": false,
    "restricted_by_viewer": null,
    "country_block": false,
    "external_url": "https://linkin.bio/uva_amsterdam",
    "external_url_linkshimmed": "https://l.instagram.com/?u=https%3A%2F%2Flinkin.bio%2Fuva_amsterdam\u0026e=ATOBo7L11uPBpsMfd6-pFnoBRaF3T-6ovlD9Blc2q1LGUjnmyuGutPfuK-ib70Bt_YmGu6cDNCX1Y1lC\u0026s=1",
    "edge_followed_by": {
        "count": 42241
    },
    "fbid": "17841401222133463",
    "followed_by_viewer": false,
    "edge_follow": {
        "count": 362
    },
    "follows_viewer": false,
    "full_name": "UvA: University of Amsterdam",
    "id": "1501672737",
    "is_business_account": true,
    "is_joined_recently": false,
    "business_category_name": "Professional Services",
    "overall_category_name": null,
    "category_enum": "UNIVERSITY",
    "category_name": null,
    "profile_pic_url": "https://scontent-amt2-1.cdninstagram.com/v/t51.2885-19/s150x150/117066908_1128864954173821_2797787766361156925_n.jpg?_nc_ht=scontent-amt2-1.cdninstagram.com\u0026_nc_ohc=PXsEzg-CKaUAX8dEtNL\u0026tp=1\u0026oh=86bb46d8006b77db2037955187e69de1\u0026oe=6056619F",
    "username": "uva_amsterdam",
    "connected_fb_page": null
}
```

Can be loaded into Python as a dictionary:
```python
{
    'biography': 'Welcome to the UvA ❌❌❌ \nFind out more about our:\n🏛 campuses 🎓 education 🔎 research\nShare your 📸 using: #uva_amsterdam\nQuestions? Contact us:',
     'blocked_by_viewer': False,
     'restricted_by_viewer': None,
     'country_block': False,
     'external_url': 'https://linkin.bio/uva_amsterdam',
     'external_url_linkshimmed': 'https://l.instagram.com/?u=https%3A%2F%2Flinkin.bio%2Fuva_amsterdam&e=ATOBo7L11uPBpsMfd6-pFnoBRaF3T-6ovlD9Blc2q1LGUjnmyuGutPfuK-ib70Bt_YmGu6cDNCX1Y1lC&s=1',
     'edge_followed_by': {'count': 42241},
     'fbid': '17841401222133463',
     'followed_by_viewer': False,
     'edge_follow': {'count': 362},
     'follows_viewer': False,
     'full_name': 'UvA: University of Amsterdam',
     'id': '1501672737',
     'is_business_account': True,
     'is_joined_recently': False,
     'business_category_name': 'Professional Services',
     'overall_category_name': None,
     'category_enum': 'UNIVERSITY',
     'category_name': None,
     'profile_pic_url': 'https://scontent-amt2-1.cdninstagram.com/v/t51.2885-19/s150x150/117066908_1128864954173821_2797787766361156925_n.jpg?_nc_ht=scontent-amt2-1.cdninstagram.com&_nc_ohc=PXsEzg-CKaUAX8dEtNL&tp=1&oh=86bb46d8006b77db2037955187e69de1&oe=6056619F',
     'username': 'uva_amsterdam',
     'connected_fb_page': None
}
```

The main differences between dictionaries in Python and the JSON file notation are:

* Python dictionaries exist in memory in Python, they are an abstract datatype. JSON is a data format and can be saved on your computer, or be transmitted as string (e.g. for a website request, sending data).
* Keys in JSON can only be of type string. This means that writing a Python dictionary with integers as keys will transform them to string. Reading back the file will therefore give you a Python dictionary with strings as keys.
* All non-ascii characters are escape sequences (e.g. `\u274c`) for ❌. This is the same for letters with diacritics (e.g. é, ê, ç, ñ). If all characters are escaped this way, you don't have to specify an encoding when opening json files.
* `True` and `False` are lowercased: `true` and `false`. `None` is `null`. 
* JSON only allows double quotes for its "strings". 

The built-in json module of Python needs to be imported first, to work with json files and notation. 

In [ ]:
import json

Let's read a json file from our disk using `json.load()`. The file comes from the public API of the municipality of Amsterdam to look up information on houses by searching on street name and house number. See: https://api.data.amsterdam.nl/atlas/search/adres/. Most often, information from such API's or 'REST-services' is given back in JSON. 

In [ ]:
with open('data/bg1.json') as jsonfile:
    data = json.load(jsonfile)

Then, we can inspect the loaded data as a Python dictionary:

In [ ]:
print(type(data))
data

When we are only interested in the information on the building, we can take out that part to store it separately. This is the first dictionary element in the list that can be found under key `data['results']`. The rest of the information is feedback from the API, telling us that there is 1 hit. 

In [ ]:
data_selection = data['results'][0]

# Delete all keys starting with an _underscore

for k in list(data_selection.keys()):
    if k.startswith('_'):
        del data_selection[k]

data_selection

Then, save it back to a json file using `json.dump()`:

In [ ]:
with open('stuff/bg1-selection.json', 'w') as outfile:
    json.dump(data_selection, outfile, indent=4)

### Quiz

* Modify that function you previously built to generate statistics for a file once more so that it returns a python dictionary with these statistics.
* Write a function that uses the `os.walk()` or `os.listdir()` method to run the file statistics function over every file in a folder. Create a dictionary that takes the file name as key, and the returned statistics dictionary as value.
* Also add arguments for a `target_file_path`, and a `data` dictionary to that function. Use the `json.dump()` method to write the dictionary to the provided file path using a with statement.
* Inspect the file by opening it on your computer with a text editor of some sorts. Find a way to make it 'pretty printed' (e.g. with _indents_). 

In [ ]:
# Your code here

source_folder = "data/gutenberg-extension"
target_file_path = "stuff/gutenberg-statistics.json"

def your_modified_statistics_function(file_path):
    # Your code
    
    return statistics_dict

def your_functions_here():
    return


---

## CSV (in Pandas)

The other often used file type is CSV (Comma Separated Values), or variants, such as TSV (Tab Separated Values). Python includes another built-in module to deal with these files: the `csv` module. But, we will be using the `Pandas`, the go-to package for data analysis, that you already imported in Notebook 0. 

A CSV file is similar to an Excel or Google Docs spreadsheet, but more limited in markup and functionality (e.g. you cannot store Excel functions). It is just a text file in which individual entries correspond to lines, and columns are separated by a comma. You can always open a CSV file with a text editor, and this also makes it so easy to store and share data with.

For the rest of the notebook we will see how to work with the two main data types in `pandas`: the `DataFrame` and a `Series`.

Information on functions and modules of Pandas cannot be found in the Python manual online, as it is an external package. Instead, you can refer to https://pandas.pydata.org/pandas-docs/stable/index.html .

### `DataFrame`


What is a `pandas.DataFrame`? 

A `DataFrame` is a collection of `Series` having the same length and whose indexes are in sync. A *collection* means that each column of a dataframe is a series

We first have to import the package. It's a convention to do this like so with Pandas, which makes the elements from this package (classes, functions, methods) available under its abbreviation `pd`:

In [ ]:
import pandas as pd

Next is loading the data. The following data comes from Wikipedia and was [automatically](https://query.wikidata.org/#%0ASELECT%20DISTINCT%20%3FmovieLabel%20%3Fimdb%20%28MIN%28%3FpublicationYear%29%20as%20%3Fyear%29%20%28year%28%3Fdate%29%20as%20%3Faward_year%29%20%28group_concat%28DISTINCT%20%3FdirectorLabel%3Bseparator%3D%22%2C%20%22%29%20as%20%3Fdirectors%20%29%20%28group_concat%28DISTINCT%20%3FcompanyLabel%3Bseparator%3D%22%2C%20%22%29%20as%20%3Fcompanies%29%20%3Fmale_cast%20%3Ffemale_cast%20WHERE%20%7B%0A%20%20%0A%20%20%7B%0A%20%20%3Fmovie%20p%3AP166%20%3Fawardstatement%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP345%20%3Fimdb%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP577%20%3Fpublication%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP57%20%3Fdirector%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP272%20%3Fcompany%20%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP31%20wd%3AQ11424%20.%0A%20%20%0A%20%20%3Fawardstatement%20ps%3AP166%20wd%3AQ102427%20%3B%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20pq%3AP585%20%3Fdate%20.%0A%20%20%7D%0A%20%20%0A%20%20BIND%28year%28%3Fpublication%29%20as%20%3FpublicationYear%29%0A%20%20%0A%20%20%7B%0A%20%20%20%20%20SELECT%20%3Fmovie%20%28COUNT%28%3Fcast_member%29%20AS%20%3Fmale_cast%29%20WHERE%20%7B%0A%20%20%20%20%20%20%3Fmovie%20wdt%3AP161%20%3Fcast_member%20.%0A%20%20%20%20%20%20%3Fcast_member%20wdt%3AP21%20wd%3AQ6581097%20.%0A%20%20%20%20%7D%20GROUP%20BY%20%3Fmovie%0A%7D%20%7B%0A%20%20%20%20SELECT%20%3Fmovie%20%28COUNT%28%3Fcast_member%29%20AS%20%3Ffemale_cast%29%20WHERE%20%7B%0A%20%20%20%20%20%20%3Fmovie%20wdt%3AP161%20%3Fcast_member%20.%0A%20%20%20%20%20%20%3Fcast_member%20wdt%3AP21%20wd%3AQ6581072%20.%0A%20%20%20%20%7D%20GROUP%20BY%20%3Fmovie%0A%20%20%7D%0A%20%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20%0A%20%20%20%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22%20.%0A%20%20%20%20%3Fmovie%20rdfs%3Alabel%20%3FmovieLabel%20.%0A%20%20%20%20%3Fdirector%20rdfs%3Alabel%20%3FdirectorLabel%20.%0A%20%20%20%20%3Fcompany%20rdfs%3Alabel%20%3FcompanyLabel%20.%20%0A%20%20%7D%0A%7D%20%0A%0AGROUP%20BY%20%3FmovieLabel%20%3Fimdb%20%3Fdate%20%3Fmale_cast%20%3Ffemale_cast%0AORDER%20BY%20%3Fyear%20) retreived. It is an overview of all movies that have won an Academy Award for Best Picture, including some extra data for the movie, including a link to the IMDB, the publication and award year, the director(s), production company and the number of male and female actors in the cast. It can be that this data is incorrect, because this information is not entered in Wikipedia. 

You can find this file in `data/academyawards.csv`. Download it from the repository and save it in the data folder if you don't have it. 

Reading in a csv with pandas is easy. We call the `pd.read_csv()` function with the file path as argument. Pandas takes care of opening and closing the file, so a `with` statement is not needed. The contents of the csv file are then read in a Pandas DataFrame object. We can store this in the variable `df`. 

Calling this variable in a Jypyter Notebook gives back a nicely formatted table with the first and last 5 rows of the file. 

In [ ]:
df = pd.read_csv('data/academyawards.csv', encoding='utf-8')
df

Think of a `DataFrame` as an in-memory spreadsheet that you can analyse and manipulate programmatically. Or, think of it as a table in which every line is a data entry, and every column holds specific information on this data.

These columns can also be seen as lists of values. They are ordered and the index of an element corresponds with the index of the data entry. The collection of all such columns is what makes the DataFrame. One column in a table is represented by a Pandas `Series`, which collects observations about a given variable. Multiple columns are a `DataFrame`. A DataFrame therefore is a collection of lists (=columns), or `Series`.

If you look for other methods on `pd` you can call, you'll also see that there is an `pd.read_excel()` option to read spreadsheets in `.xls` or `.xlsx`. You can also  use this, if you have these kind of files.

### Statistics
Now that we loaded our DataFrame, we can make pandas print some statistics on the file. 

In [ ]:
df.head()  # First 5 rows

In [ ]:
df.tail()  # Last 5 rows

In [ ]:
df.describe()  # Descriptive statistics

As you can see by what they return, these methods return another DataFrame with some descriptive statistics on the file, such as the number of entries (count), the mean of the numerical values, the standard deviation, minimum and maximum values, and the 25th, 50th, and 75th percentiles.

The `.info()` method can also be informative. It gives you information about a dataframe:
- how much space does it take in memory?
- what is the datatype of each column?
- how many records are there?
- how many `null` values does each column contain (!)?

In [ ]:
df.info()

Pandas automatically interprets which datatypes are used in the file, but this is not always correct. Especially if you have empty fields in the DataFrame, any other integers get interpreted as float. Every column has one datatype. You can check them separately by requesting the `.dtypes` argument on the `df`. 

The 'object' type is a string in this file, 'int64' is an integer. 

In [ ]:
df.dtypes

We expect different datatypes for the description-dataframe:

In [ ]:
description_df = df.describe()
description_df.dtypes

### Slicing and selecting

#### `df['column1']`
You can select a single column by calling this column name as if the DataFrame was a dictionary. A single column from a DataFrame returns a `Series` object. 

In [ ]:
print(type(df['movie']))

df['movie']

The `Series` object is very similar to a `list`:

In [ ]:
movies = df['movie']

print("Length:", len(movies))
print()

for n, movie in enumerate(movies[:10], 1):
    print(n, movie, sep='\t')

#### `df[['column1', 'column2']]`
We can also slice a DataFrame by calling multiple column names as one list:

In [ ]:
df[['movie', 'imdb']]

### Looping over DataFrames

#### `zip(df['column1', df['column2')`
Going over these items in a `for` loop needs a different approach. The built-in `zip()` function ([manual](https://docs.python.org/3/library/functions.html#zip)) takes two iterables of even length and creates a new iterable of tuples. The number of arguments/iterables that you give to `zip()` determines the length of the tuples.

In [ ]:
n = 0
for movie, imdb in zip(df['movie'], df['imdb']):
    
    if n > 9:
        break  # stop flooding the Notebook
    
    print(movie, "http://www.imdb.com/title/" + imdb, sep='\t')
    n += 1

#### `.to_dict(orient='record')`
Or, accessing all entries in a convenient way, as a python dictionary for instance, can be done with the `.to_dict(orient='records')` method:

In [ ]:
for r in df.to_dict(orient='records'):

    name = r['movie']
    year = r['year']
    won = r['award_year']
    
    print("The movie " + name + " was produced in " + str(year) + " and won in " + str(won) + ".")
    
    print()
    
    break  # To not flood the notebook, only print the first

#### `.iterrows()`
Or you can use the `.iterrows()` method, which gives you tuples of the index of the row, and the row itself as `Series` object:

In [ ]:
for n, r in df.iterrows():
    
    name = r.movie  # You can use a dot notation here
    year = r.year
    won = r.award_year
    
    print("The movie " + name + " was produced in " + str(year) + " and won in " + str(won) + ".")
    
    print()
    
    break  # To not flood the notebook, only print the first

---

### Analysis

You already saw above that you could get statistics by calling `.describe()` on a DataFrame. You can also get these metrics for individual columns. Let's ask the maximum number of male and female actors in the cast of a movie:

In [ ]:
df['female_cast'].max()

In [ ]:
df['male_cast'].max()

You can also apply these operations to multiple columns at once. You get a `Series` object back. 

In [ ]:
df[['male_cast', 'female_cast']].max()

To find the corresponding movie title, we can ask Pandas to give us the record in which these maxima occur. This is done through `df.loc`. This works by asking: "Give me all the locations (=rows) for which a value in a specified column is equal to this value". 

In [ ]:
for column_name, value in df[['male_cast', 'female_cast']].max().items():
    
    print("Movie with maximum for", column_name)
    row = df.loc[df[column_name] == value]
    print(row.movie)
    
    print()

Other functions that can be used are for instance `.mean()`, `.median()`, `.std()` and `.sum()`.

In [ ]:
df['female_cast'].mean()

In [ ]:
df['male_cast'].mean()

In [ ]:
df['female_cast'].sum()

In [ ]:
df['male_cast'].sum()

Pandas also understands dates, but you have to tell it to interpret a column as such. We can change the `year` column in-place so that it is not interpreted as integer, but as a date object. 

In this case, since we only have the year available, and not a full date such as `2021-02-22` (YYYY-mm-dd), we have to specify the format. Typing `%Y` as string is shorthand for `YYYY`. It returns a full date, so every month and day are set to January first. 

In [ ]:
df['year'] = pd.to_datetime(df['year'], format='%Y')
df['award_year'] = pd.to_datetime(df['award_year'], format='%Y')

In [ ]:
df['year']

### Plotting

Let's try to make some graphs from our data, for instance the number of male/female actors over time. 

We now have a year column that is interpreted as time by Pandas. These values can figure as values on a x-axis in a graph. The y-axis would then give info on the number of male and female actors in the movie. 

First, we set an **index** for the DataFrame. This determines how the data can be accessed. Normally, this is a range of 0 untill the number of rows. But, you can change this, so that we can analyse the dataframe on a time index. 

In [ ]:
# Select only what we need
df_actors = df[['award_year', 'male_cast', 'female_cast']]
df_actors

In [ ]:
df_actors = df_actors.set_index('award_year')
df_actors

Then simply call `.plot()` on your newly created DataFrame!

In [ ]:
df_actors.plot(figsize=(15,10))

There are tons of parameters, functions, methods, transformations you can use on DataFrames and also on this plotting function. Luckily, plenty of guides and examples can be found on the internet.

### Grouping

Some directors have won multiple Oscars. To find out which, we have to count the number of rows in the DataFrame that include the same director. There is a Pandas function for this: `.count()`. Calling this on the DataFrame itself would give us the total number of rows only, per column. Therefore, we have to tell Pandas that we want to group by a particular column, say 'directors'.

In [ ]:
df.groupby('directors')

It does not give back something nicely formatted or interpretable. It's just another Python object. Let's call the `.count()` on this object:

In [ ]:
df.groupby('directors').count()

Remember that this counts the numer of rows. As we know that each row is one movie, we can trim this down to:

In [ ]:
director_counts = df.groupby('directors').count()['movie']
director_counts

Now, get all directors that have won an Oscar more than once by specifying a conditional operator:

In [ ]:
director_counts[director_counts > 1]

### Adding a column

If we want to get the total number of actors per movie, we have to sum the values from the `male_cast` and `female_cast` columns. 

You can do this in a for loop, by going over every row (like we saw above), but you can also sum the individual columns. Pandas will then add up the values with the same index and will return a new Series of the same length with the values summed. 

In [ ]:
total_cast = df['male_cast'] + df['female_cast']
total_cast

Then, we add it as a column in our original dataframe. The only requirement for adding a column to a DataFrame is that the length of the Series or list is the same as that of the DataFrame. 

In [ ]:
df['total_cast'] = total_cast
df

Optionally, we can sort the DataFrame by column. For instance, from high to low (`ascending=False`) for the newly created `total_cast` column.

In [ ]:
df_sorted = df.sort_values('total_cast', ascending=False)

### Saving back the file

Use one of the `.to_csv()` or `.to_excel` functions to save the DataFrame. Again, no `with` statement needed, just a file path (and an encoding).

In [ ]:
df_sorted.to_csv('stuff/academyawards_sum.csv', encoding='utf-8')

You need to specify `index=False` if you want to prevent a standard index (0,1,2,3...) to be saved in the file as well. 

In [ ]:
df_sorted.to_csv('stuff/academyawards_sum.csv', encoding='utf-8', index=False)

Open the contents in Excel, LibreOffice Calc, or another program to read spreadsheets!

---

# Exercises

### Exercise 1 (previously Exercise 6 in Notebook 2)

Read the file `data/adams-hhgttg.txt` and:

- Count the number of lines in the file

- Count the number of non-empty lines

- Read each line of the input file, remove its newline character and write it to file `stuff/adams-output.txt`

- Compute the average number of alphanumeric characters per line

- Identify all the unique words used in the text (no duplicates!) and write them in a text file called `stuff/lexicon.txt` (one word per line)

In [ ]:
# your code here

with open("stuff/lexicon.txt", "w") as infile:
    infile.write("something")

### Exercise 2

TBD